# Readme


* This notebook can be either run with "Cell" - "Run All", or with "Cell" - "Run Cells" (from the above menu)
  * With the first option ("Run All"), the entire script will be automatically executed till the end.
  * With the second option ("Run Cells"), only the code of a single cell will be executed. Clicking THIS it is possibile to procede, cell by cell, till the end of the code.
  
 ![skipforward](https://raw.githubusercontent.com/antonelepfl/testvue/master/skip_next.png "size=10")


* Each time a notebook is executed, a kernel status symbol is displayed in the top right corner of the notebook
  * When there is a circle bullet •, it means that the kernel is running and the user have to be sure to not interfere with the code execution
  * When there is an empty circle bullet ○, it means that the kernel is idle and the user can interfere with the notebook.
  
See Guidebook at https://collab.humanbrainproject.eu/#/collab/1655/nav/18580

## Hint for this notebook

Since the simulations are computational intensive, this optimization can be run on the Collab platform with a maximum of 4 individuals for only 4 generations.

### Preparation
#### Define the Simulation parameters

Define the number of individuals and the number of generations.

In [ ]:
!ln -sfn /home/jovyan/.local/nrn-7.6/ /home/jovyan/.local/nrn

In [ ]:
!pip3 install pebble >/dev/null

In [ ]:
NUMBER_INDIVIDUALS = 4
NUMBER_GENERATIONS = 4

In [ ]:
# Installation of BluePyOpt
!pip3 install bluepyopt --upgrade --no-dependencies >/dev/null

This custom class define the parameters and section to create a custom built axon.
It is used to define the temperature, V_init and the tables used in the MOD files too.

In [ ]:
import bluepyopt as bpop
import bluepyopt.ephys as ephys
import matplotlib.pyplot as plt
import os


class GrcMonoMorphology(ephys.morphologies.NrnFileMorphology):
  def __init__(self, morphology_path, do_replace_axon):
    super(GrcMonoMorphology, self).__init__(morphology_path, do_replace_axon)
    
  @staticmethod
  def replace_axon(sim=None, icell=None):
        """Replace axon"""

        # Define origin of distance function
        sim.neuron.h.distance(sec=icell.soma[0])
        
        #Temperature and v_init
        sim.neuron.h.celsius = 30
        sim.neuron.h.v_init = -80
        
        #Ionic channels tables
        sim.neuron.h.usetable_GrG_Na = 0
        sim.neuron.h.usetable_GrC_pNa = 0
        sim.neuron.h.usetable_GrC_CaHVA = 0
        sim.neuron.h.usetable_GrG_KV = 0
        sim.neuron.h.usetable_GrC_KA = 0
        sim.neuron.h.usetable_GrC_Kir = 0
        sim.neuron.h.usetable_GrC_KCa = 0
        sim.neuron.h.usetable_GrG_KM = 0


In [ ]:
!pip3 install --upgrade --no-dependencies "hbp-service-client==1.1.1" >/dev/null

In [ ]:
#access to the collaboratory storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()

To run the optimization, it is necessary to download specific files, from the collab storage space, to the place where the python notebook will run. 
The files downloaded are: the neuron morphology (`asc` - neurolucida format) and the ionic channels (`mod` files).

In [ ]:
collab_storage_path = '/9135'

clients.storage.download_file(collab_storage_path + '/Single cells usecases/Granule cell mono section/granule_mono.zip', 'granule_mono.zip')

In [ ]:
%%capture
!unzip -o granule_mono.zip

This command will compile the `MOD` files

In [ ]:
!nrnivmodl >/dev/null 2>/dev/null

#### Defining passive properties

First step in the definition of the BluePyOpt Granular cell template.
- The morphology, which can be a neurolucida file or an swc file.
- The locations for the ionic channels.

In [ ]:
morph = GrcMonoMorphology('GrCmorphmono.asc', do_replace_axon = True) 
somatic_loc = ephys.locations.NrnSeclistLocation(\
    'somatic', seclist_name='somatic')

This cell contains all the information to place the passive properties, with specific parameters, on the morphological locations previously defined. 

* `name`: Name of the property, which must be unique
* `param_name`: Name as taken from the `mod` file
* `value`: The value of the property
* `locations`: The location of where this property is being used, for example soma (somatic_loc).
* `frozen`: Indicating this property has a fixed value (`frozen=true`) or is subject to optimization (`frozen=false`).

In [ ]:
#Passive properties 
cm_param_soma = ephys.parameters.NrnSectionParameter(
        name='cm_soma',
        param_name='cm',
        value=1.0,
        locations=[somatic_loc],
        frozen=True)

ena = ephys.parameters.NrnSectionParameter(
        name='ena_soma',
        param_name='ena',
        value=87.39,
        locations=[somatic_loc],
        frozen=True)

ek = ephys.parameters.NrnSectionParameter(
        name='ek_soma',
        param_name='ek',
        value=-84.69,
        locations=[somatic_loc],
        frozen=True)

el = ephys.parameters.NrnSectionParameter(                            
        name='el_soma',
        param_name='el_GrC_Lkg1',
        value=-58,
        locations=[somatic_loc],
        frozen=True)

egaba = ephys.parameters.NrnSectionParameter(                         
        name='egaba_soma',
        param_name='egaba_GrC_Lkg2',
        value=-65,
        locations=[somatic_loc],
        frozen=True)

eca = ephys.parameters.NrnSectionParameter(
        name='eca_soma',
        param_name='eca',
        value=129.33,
        locations=[somatic_loc],
        frozen=True)

Ra = ephys.parameters.NrnSectionParameter(
        name='Ra_soma',
        param_name='Ra',
        value=100,
        locations=[somatic_loc],
        frozen=True)

#ionic mechanisms
Mn_Na = ephys.mechanisms.NrnMODMechanism(
        name='Na_mono',
        prefix='GrG_Na',
        locations=[somatic_loc])

Mn_Nar = ephys.mechanisms.NrnMODMechanism(
        name='Nar_mono',
        prefix='GrG_Nar',
        locations=[somatic_loc])

Mn_Nap = ephys.mechanisms.NrnMODMechanism(
        name='Nap_mono',
        prefix='GrC_pNa',
        locations=[somatic_loc])

Mn_leak1 = ephys.mechanisms.NrnMODMechanism(
        name='Leak1_mono',
        prefix='GrC_Lkg1',
        locations=[somatic_loc])

Mn_leak2 = ephys.mechanisms.NrnMODMechanism(
        name='Leak2_mono',
        prefix='GrC_Lkg2',
        locations=[somatic_loc])

Mn_Ca = ephys.mechanisms.NrnMODMechanism(
        name='Ca_mono',
        prefix='GrC_CaHVA',
        locations=[somatic_loc])

Mn_Calc = ephys.mechanisms.NrnMODMechanism(      
        name='Calc_mono',
        prefix='Calc',
        locations=[somatic_loc])

Mn_Kir = ephys.mechanisms.NrnMODMechanism(
        name='Kir_mono',
        prefix='GrC_Kir',
        locations=[somatic_loc])

Mn_Kv = ephys.mechanisms.NrnMODMechanism(
        name='Kv_mono',
        prefix='GrG_KV',
        locations=[somatic_loc])

Mn_Kca = ephys.mechanisms.NrnMODMechanism(
        name='Kca_mono',
        prefix='GrC_KCa',
        locations=[somatic_loc])

Mn_Ka = ephys.mechanisms.NrnMODMechanism(
        name='Ka_mono',
        prefix='GrC_KA',
        locations=[somatic_loc])

Mn_km = ephys.mechanisms.NrnMODMechanism(
        name='km_mono',
        prefix='GrG_KM',
        locations=[somatic_loc])

This is the section in which the conductance range for each ionic channel is defined. 

In [ ]:
#parameters

Mn_Na_param = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GrG_Na_mono',
        param_name='gnabar_GrG_Na',
        locations=[somatic_loc],
        bounds=[0.0104, 0.0156],
        frozen=False)

Mn_Nar_param = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GrG_Nar_mono',
        param_name='gnabar_GrG_Nar',
        locations=[somatic_loc],
        bounds=[0.0004, 0.0006],
        frozen=False)

Mn_Nap_param = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GrC_pNa_mono',
        param_name='gnabar_GrC_pNa',
        locations=[somatic_loc],
        bounds=[1.60e-5, 2.40e-5],
        frozen=False)

Mn_leak1_param = ephys.parameters.NrnSectionParameter(                                    
        name='gl_GrC_Lkg1_mono',
        param_name='gl_GrC_Lkg1',
        locations=[somatic_loc],
        bounds=[4.54e-5, 6.82e-5],
        frozen=False)

Mn_leak2_param = ephys.parameters.NrnSectionParameter(                                    
        name='egaba_GrC_Lkg2_mono',
        param_name='egaba_GrC_Lkg2',
        locations=[somatic_loc],
        bounds=[1.74e-5, 2.60e-5],
        frozen=False)

Mn_Ca_param = ephys.parameters.NrnSectionParameter(
        name='gcabar_GrC_CaHVA_mono',
        param_name='gcabar_GrC_CaHVA',
        locations=[somatic_loc],
        bounds=[0.000368, 0.000552],
        frozen=False)

Mn_Calc_param = ephys.parameters.NrnSectionParameter(           
        name='beta_Calc_mono',
        param_name='beta_Calc',
        locations=[somatic_loc],
        bounds=[1.5, 1.51],
        frozen=False)

Mn_Kir_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_GrC_Kir_mono',
        param_name='gkbar_GrC_Kir',
        locations=[somatic_loc],
        bounds=[0.00072, 0.00108],
        frozen=False)

Mn_Kv_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_GrG_KV_mono',
        param_name='gkbar_GrG_KV',
        locations=[somatic_loc],
        bounds=[0.0024, 0.0036],
        frozen=False)

Mn_Kca_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_GrC_KCa_mono',
        param_name='gkbar_GrC_KCa',
        locations=[somatic_loc],
        bounds=[0.0032, 0.0048],
        frozen=False)

Mn_Ka_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_GrC_KA_mono',
        param_name='gkbar_GrC_KA',
        locations=[somatic_loc],
        bounds=[0.0032, 0.0048],
        frozen=False)

Mn_km_param = ephys.parameters.NrnSectionParameter(
        name='gkbar_GrG_KM_mono',
        param_name='gkbar_GrG_KM',
        locations=[somatic_loc],
        bounds=[0.00028, 0.00042],
        frozen=False)

#### Creating the template

To create the cell template, we pass all the previously prepared objects to the constructor of the template.
The section lists for the axon are declared here and used in the morphology class.

In [ ]:
grcmono = ephys.models.CellModel(                     
        name='grcmono',
        morph=morph,
        mechs=[Mn_Na,
               Mn_Nar, 
               Mn_Nap, 
               Mn_leak1, 
               Mn_leak2, 
               Mn_Ca, 
               Mn_Calc, 
               Mn_Kir, 
               Mn_Kv, 
               Mn_Kca, 
               Mn_Ka, 
               Mn_km],

        params=[cm_param_soma, 
                ena, 
                ek, 
                el,
                egaba,   
                eca, 
                Ra, 
                Mn_Na_param, 
                Mn_Nar_param, 
                Mn_Nap_param, 
                Mn_leak1_param , 
                Mn_leak2_param , 
                Mn_Ca_param, 
                Mn_Calc_param, 
                Mn_Kir_param, 
                Mn_Kv_param, 
                Mn_Kca_param, 
                Mn_Ka_param, 
                Mn_km_param]) 
        
print(grcmono)

Placing the electrode for the current injections.

In [ ]:
soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)

#### Defining the stimuli, recordings location and protocols. 

For each protocol there is a recording location and a stimulus; in this example both are placed on the soma.

The delay, duration and total duration can be changed, but a longer simumation time also requires more computational time.

In [ ]:
sweep_protocols = []
for protocol_name, amplitude in [('step1', 0.01), ('step2', 0.016), ('step3', 0.022)]:#
    stim = ephys.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=2000,
                location=soma_loc,
                total_duration=2200)
    rec = ephys.recordings.CompRecording(
            name='%s.soma.v' % protocol_name,
            location=soma_loc,
            variable='v')
    protocol = ephys.protocols.SweepProtocol(protocol_name, [stim], [rec])
    sweep_protocols.append(protocol)
threestep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)

Loading NEURON as the simulator

In [ ]:
nrn = ephys.simulators.NrnSimulator()

These conductances are based on the original non-optimized model to give an idea of how the optimized cells have to respond to be declared valid.

In [ ]:
default_params = {'gnabar_GrG_Na_mono': 0.013, 
                  'gnabar_GrG_Nar_mono': 0.0005, 
                  'gnabar_GrC_pNa_mono': 2e-5, 
                  'gl_GrC_Lkg1_mono': 5.68e-5,
                  'egaba_GrC_Lkg2_mono': 2.17e-5,
                  'gcabar_GrC_CaHVA_mono': 0.00046,
                  'beta_Calc_mono': 1.5,
                  'gkbar_GrC_Kir_mono': 0.0009,
                  'gkbar_GrG_KV_mono': 0.003,
                  'gkbar_GrC_KCa_mono': 0.004,
                  'gkbar_GrC_KA_mono': 0.004,
                  'gkbar_GrG_KM_mono': 0.00035}

responses = threestep_protocol.run(cell_model=grcmono, param_values=default_params, sim=nrn)

#### Plotting the response traces

Three traces are plotted in a graph, each trace generated by a different current injection step.

In [ ]:
def plot_responses(responses):
    plt.subplot(3,1,1)
    plt.plot(responses['step1.soma.v']['time'], responses['step1.soma.v']['voltage'], label='step1')
    plt.legend()
    plt.subplot(3,1,2)
    plt.plot(responses['step2.soma.v']['time'], responses['step2.soma.v']['voltage'], label='step2')
    plt.legend()
    plt.subplot(3,1,3)
    plt.plot(responses['step3.soma.v']['time'], responses['step3.soma.v']['voltage'], label='step3')
    plt.legend()
    plt.tight_layout()

plot_responses(responses)
plt.show()


List of all the features to be used for each step of current injection.
All these information were taken from in vitro traces obtain from experiments on mice.

In [ ]:
efel_feature_means = {'step1': {'AP_height': 20.93,
                                'ISI_CV':0.261,
                                'AHP_depth_abs_slow':-52.69,
                                'AP_width':0.665,
                                'voltage_base':-68.5,
                                'AHP_depth_abs':-59.21,
                                'time_to_first_spike':31.9,
                                'mean_frequency': 55.25}, 

                      'step2': {'AP_height': 19.255,
                                'ISI_CV':0.14,
                                'AHP_depth_abs_slow':-48.935,
                                'AP_width':0.695,
                                'voltage_base':-68.77,
                                'AHP_depth_abs':-58.3,
                                'time_to_first_spike':19.0,
                                'mean_frequency': 84.36}, 
                    
                      'step3': {'AP_height': 17.645,
                                'ISI_CV':0.148,
                                'AHP_depth_abs_slow':-32.67,
                                'AP_width':0.7135,
                                'voltage_base':-69.125,
                                'AHP_depth_abs':-57.191,
                                'time_to_first_spike':14.65,
                                'mean_frequency': 109.8}}

#### Objectives

The function that try to match what was defined above - for example the speed, with the actual result in the individual obtained during each generation.

As per BluePyOpt default, each feature will be an objective.

In [ ]:
objectives = []

for protocol in sweep_protocols:
    stim_start = protocol.stimuli[0].step_delay
    stim_end = stim_start + protocol.stimuli[0].step_duration
    for efel_feature_name, mean in efel_feature_means[protocol.name].items():
        feature_name = '%s.%s' % (protocol.name, efel_feature_name)
        feature = ephys.efeatures.eFELFeature(
                    feature_name,
                    efel_feature_name=efel_feature_name,
                    recording_names={'': '%s.soma.v' % protocol.name},
                    stim_start=stim_start,
                    stim_end=stim_end,
                    exp_mean=mean,
                    exp_std=0.05 * mean)
        objective = ephys.objectives.SingletonObjective(
            feature_name,
            feature)
        objectives.append(objective)

#### Objective evaluator

Calculating the quality of each results.

In [ ]:
        
score_calc = ephys.objectivescalculators.ObjectivesCalculator(objectives)

#### Cell Evaluator

The conductances that will be evaluated during the optimization

In [ ]:
cell_evaluator = ephys.evaluators.CellEvaluator(
        cell_model=grcmono,
        param_names=['gnabar_GrG_Na_mono',
                     'gnabar_GrG_Nar_mono',
                     'gnabar_GrC_pNa_mono', 
                     'gl_GrC_Lkg1_mono',
                     'egaba_GrC_Lkg2_mono',
                     'gcabar_GrC_CaHVA_mono',
                     'beta_Calc_mono',
                     'gkbar_GrC_Kir_mono',
                     'gkbar_GrG_KV_mono',
                     'gkbar_GrC_KCa_mono',
                     'gkbar_GrC_KA_mono',
                     'gkbar_GrG_KM_mono'],
        fitness_protocols={threestep_protocol.name: threestep_protocol},
        fitness_calculator=score_calc,
        sim=nrn)

#### Optimization

Number of offsprings and max generation.

In [ ]:
print(cell_evaluator.evaluate_with_dicts(default_params))


optimisation = bpop.optimisations.DEAPOptimisation(
    evaluator=cell_evaluator,
    offspring_size = NUMBER_INDIVIDUALS,
    seed=os.getenv('BLUEPYOPT_SEED'))

final_pop, hall_of_fame, logs, hist = optimisation.run(max_ngen=NUMBER_GENERATIONS)

Evaluation of the best individual based on its simulations.

In [ ]:
print('Final population:', final_pop)
import numpy as np

best_ind = hall_of_fame[0]
print('Best individual: ', best_ind)
print('Fitness values: ', best_ind.fitness.values)

best_ind_dict = cell_evaluator.param_dict(best_ind)
print(cell_evaluator.evaluate_with_dicts(best_ind_dict))

plot_responses(threestep_protocol.run(cell_model=grcmono, param_values=best_ind_dict, sim=nrn))

Plot of the best individual and the minimum fitness obtained during the optimization.

In [ ]:
#import numpy
gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
plt.plot(gen_numbers, min_fitness, label='min fitness')
plt.xlabel('generation #')
plt.ylabel('score (# std)')
plt.legend()
plt.xlim(min(gen_numbers) - 1, max(gen_numbers) + 1) 
plt.ylim(0.9*min(min_fitness), 1.1 * max(min_fitness)) 
plt.show()
 